In [1]:
import sys
sys.path.append('../')

In [2]:
import libs.db
import libs.prep_df
import pandas as pd
import sqlite3

In [3]:
sql_file = "sp10_historical.db"
conn = libs.db.create_connection(sql_file)
df = libs.db.df_from_db(conn)
libs.db.close_connection(conn)
df.head()

,id,symbol,timestamp,close
0,1,AAPL,2017-01-03T09:00:00+00:00,116.05
1,2,AAPL,2017-01-03T10:00:00+00:00,116.02
2,3,AAPL,2017-01-03T11:00:00+00:00,116.28
3,4,AAPL,2017-01-03T12:00:00+00:00,116.02
4,5,AAPL,2017-01-03T13:00:00+00:00,116.20


In [4]:
norm_df = libs.prep_df.normalize_df(df)

In [5]:
tickers = libs.db.get_tickers("sp10.csv")
print(tickers)

['AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'TSLA', 'BRK.B', 'NVDA', 'JPM', 'JNJ']


In [13]:
num_ticks = 60
ticker_shifted_df = pd.DataFrame()
for ticker in tickers:
    ticker_shifted_df[ticker] = norm_df[ticker].copy()
    for i in range(1,num_ticks):
        name = f"{ticker}-{i}"
        ticker_shifted_df[name] = norm_df[ticker].shift(i)
ticker_shifted_df = ticker_shifted_df.copy()
ticker_shifted_df.shape

ipykernel_launcher:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
ipykernel_launcher:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


(17080, 600)

In [7]:
ticker_returns_df_list = []
for ticker in tickers:
    name = f"{ticker}_24_Return"
    returns_df = pd.DataFrame()
    returns_df[name] = norm_df[ticker].pct_change(periods=16).shift(-16)
    ticker_returns_df_list.append(returns_df)
len(ticker_returns_df_list)

10

In [8]:
a = [1,2,3,4,5]
b = [*a,6]
b

[1, 2, 3, 4, 5, 6]

In [9]:
pctchange_df = norm_df.pct_change()
pctchange_df = pd.concat([ticker_shifted_df, *ticker_returns_df_list], axis=1)
pctchange_df.dropna(inplace=True)
pctchange_df.describe()

,AAPL,AAPL-1,AAPL-2,AAPL-3,AAPL-4,AAPL-5,AAPL-6,AAPL-7,AAPL-8,AAPL-9,...,AAPL_24_Return,MSFT_24_Return,AMZN_24_Return,FB_24_Return,GOOGL_24_Return,TSLA_24_Return,BRK.B_24_Return,NVDA_24_Return,JPM_24_Return,JNJ_24_Return
count,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000,...,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000,17005.000000
mean,198.193261,198.193145,198.193029,198.192912,198.192798,198.192659,198.192541,198.192353,198.192175,198.191981,...,0.000797,0.001397,0.001812,0.001038,0.001009,0.002480,0.000515,0.001950,0.000712,0.000410
std,72.293373,72.293500,72.293628,72.293756,72.293881,72.294033,72.294163,72.294371,72.294569,72.294783,...,0.029793,0.017820,0.056455,0.021272,0.017175,0.046425,0.013247,0.029086,0.018518,0.012943
min,104.050000,104.050000,104.050000,104.050000,104.050000,104.050000,104.050000,104.050000,104.050000,104.050000,...,-0.750288,-0.441069,-0.872601,-0.219802,-0.127222,-0.805981,-0.108916,-0.198644,-0.149697,-0.116202
25%,150.350000,150.350000,150.350000,150.350000,150.350000,150.350000,150.350000,150.350000,150.350000,150.350000,...,-0.006934,-0.006129,-0.007460,-0.008182,-0.006672,-0.015712,-0.005608,-0.012472,-0.007462,-0.005201
50%,177.010000,177.010000,177.010000,177.010000,177.010000,177.010000,177.010000,177.010000,177.010000,177.010000,...,0.001553,0.001275,0.001575,0.001084,0.001307,0.001684,0.000600,0.002316,0.000338,0.000556
75%,220.170000,220.170000,220.170000,220.170000,220.170000,220.170000,220.170000,220.170000,220.170000,220.170000,...,0.010357,0.009423,0.010906,0.011204,0.009396,0.020302,0.006889,0.017547,0.008814,0.006613
max,513.890000,513.890000,513.890000,513.890000,513.890000,513.890000,513.890000,513.890000,513.890000,513.890000,...,0.166464,0.788871,6.868773,0.158647,0.116159,0.316360,0.118387,0.225417,0.180847,0.118699


In [10]:
y_start = len(tickers)*num_ticks

In [11]:
from sklearn.linear_model import LinearRegression
X = pctchange_df.iloc[:,0:y_start].values
y = []
for i in range(0,len(tickers)):
    y.append(pctchange_df.iloc[:,y_start+i].values)
lin_models = []
y_lin_predicted = []

In [12]:
for i in range(0,len(tickers)):
    ticker = tickers[i]
    model = LinearRegression()
    model.fit(X,y[i])
    lin_models.append(model)
    predicted = model.predict(X)
    y_lin_predicted.append(predicted)
    
    

In [16]:
from sklearn.metrics import r2_score
import numpy as np
r2 = [0]*len(tickers)
for i in range(0,len(tickers)):
    r2[i] = r2_score(y[i], y_lin_predicted[i])
    print(f"The r2 score for symbol {tickers[i]} is {r2[i]}")


The r2 score for symbol AAPL is 0.14207293518856834
The r2 score for symbol MSFT is 0.12052385553597367
The r2 score for symbol AMZN is 0.49827031996310156
The r2 score for symbol FB is 0.08025409539994288
The r2 score for symbol GOOGL is 0.09303368485812658
The r2 score for symbol TSLA is 0.10845650056927458
The r2 score for symbol BRK.B is 0.0987635134842908
The r2 score for symbol NVDA is 0.07870614592396274
The r2 score for symbol JPM is 0.11098963134706163
The r2 score for symbol JNJ is 0.09101566360039248
